In [1]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from os.path import abspath
import numpy as np
import pandas as pd
from utils.generate_network import generate_network
from utils.prepare_data import prepare_data
from utils.popphy_io import get_config, save_params, load_params
from utils.popphy_io import get_stat, get_stat_dict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from models.PopPhy import PopPhyCNN
from models.CNN1D import CNN1D
from models.MLPNN import MLPNN
from models.RF import RF
from models.SVM import SVM
from models.LASSO import LASSO
import warnings
from datetime import datetime
import json
from cyjupyter import Cytoscape
import warnings
warnings.filterwarnings("ignore")

In [2]:
config = get_config()
filt_thresh = config.get('Evaluation', 'FilterThresh')
dataset = config.get('Evaluation', 'DataSet')
num_runs = int(config.get('Evaluation', 'NumberRuns'))
num_test = int(config.get('Evaluation', 'NumberTestSplits'))
path = "../data/" + dataset

my_maps, raw_x, tree_x, raw_features, tree_features, labels, label_set, g, feature_df = prepare_data(path, config)

num_class = len(np.unique(labels))
if num_class == 2:
    metric = "AUC"
else:
    metric = "MCC"

seed = np.random.randint(100)
np.random.seed(seed)
np.random.shuffle(my_maps)
np.random.seed(seed)
np.random.shuffle(raw_x)
np.random.seed(seed)
np.random.shuffle(tree_x)
np.random.seed(seed)
np.random.shuffle(labels)

n_values = np.max(labels) + 1
labels_oh = np.eye(n_values)[labels]

tree_row = my_maps.shape[1]
tree_col = my_maps.shape[2]

print("There are %d classes...%s" % (num_class, ", ".join(label_set)))

cv_list = ["Run_" + str(x) + "_CV_" + str(y) for x in range(num_runs) for y in range(num_test)]
seeds = np.random.randint(1000, size=num_runs)

There are 269 raw features...
Building tree structure...
Found tree file...
Populating trees...
There are 479 tree features...
There are 2 classes...n, cirrhosis


In [3]:
popphy_stat_df = pd.DataFrame(index=["AUC", "MCC", "Precision", "Recall", "F1"], columns=cv_list)

feature_scores = {}

for l in label_set:
    feature_scores[l] = pd.DataFrame(index=tree_features)
run = 0
for seed in seeds:
    skf = StratifiedKFold(n_splits=num_test, shuffle=True, random_state=seed)
    fold = 0
    for train_index, test_index in skf.split(my_maps, labels):
        train_x, test_x = my_maps[train_index,:,:], my_maps[test_index,:,:]
        train_y, test_y = labels_oh[train_index,:], labels_oh[test_index,:]
        
        train_x = np.log(train_x + 1)
        test_x = np.log(test_x + 1)
        
        c_prob = [0] * len(np.unique(labels))
        train_weights = []

        for l in np.unique(labels):
            a = float(len(labels))
            b = 2.0 * float((np.sum(labels==l)))
            c_prob[int(l)] = a/b

        c_prob = np.array(c_prob).reshape(-1)

        for l in np.argmax(train_y, 1):
            train_weights.append(c_prob[int(l)])
        train_weights = np.array(train_weights)
        
        scaler = MinMaxScaler().fit(train_x.reshape(-1, tree_row * tree_col))
        train_x = np.clip(scaler.transform(train_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)
        test_x = np.clip(scaler.transform(test_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)

        train = [train_x, train_y]
        test = [test_x, test_y]

        popphy_model = PopPhyCNN((tree_row, tree_col), num_class, config)

        if fold + run == 0:
            print(popphy_model.model.summary())
            print("\n\n Run\tFold\t%s" % (metric))

        popphy_model.train(train, train_weights)
        preds, stats = popphy_model.test(test)
        if num_class == 2:
                popphy_stat_df.loc["AUC"]["Run_" + str(run) + "_CV_" + str(fold)]=stats["AUC"]
        popphy_stat_df.loc["MCC"]["Run_" + str(run) + "_CV_" + str(fold)]=stats["MCC"]
        popphy_stat_df.loc["Precision"]["Run_" + str(run) + "_CV_" + str(fold)]=stats["Precision"]
        popphy_stat_df.loc["Recall"]["Run_" + str(run) + "_CV_" + str(fold)]=stats["Recall"]
        popphy_stat_df.loc["F1"]["Run_" + str(run) + "_CV_" + str(fold)]=stats["F1"]

        if metric == "AUC":
                print("# %d\t%d\t%.3f" % (run, fold, stats["AUC"]))
        if metric == "MCC":
                print("# %d\t%d\t%.3f\t" % (run, fold, stats["MCC"]))

        scores = popphy_model.get_feature_scores(train, g, label_set, tree_features, config)
        for l in range(len(label_set)):
                score_list = scores[:,l]
                lab = label_set[l]
                feature_scores[lab]["Run_" + str(run) + "_CV_" + str(fold)] = score_list


        popphy_model.destroy()
        fold += 1
    run += 1

W0813 19:28:09.108997 139995068057408 deprecation.py:506] From /home/dreiman/anaconda3/envs/meta-signer2/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise (GaussianNois (None, 10, 179, 1)        0         
_________________________________________________________________
conv_0 (Conv2D)              (None, 8, 170, 32)        992       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 85, 32)         0         
_________________________________________________________________
flatten (Flatten)            (None, 10880)             0         
_________________________________________________________________
dropout (Dropout)            (None, 10880)             0         
_________________________________________________________________
fc_0 (Dense)                 (None, 32)                348192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0

In [4]:
popphy_stat_df

,Run_0_CV_0,Run_0_CV_1,Run_0_CV_2,Run_0_CV_3,Run_0_CV_4,Run_0_CV_5,Run_0_CV_6,Run_0_CV_7,Run_0_CV_8,Run_0_CV_9
AUC,0.9375,0.868056,0.8125,0.819444,0.916667,0.924242,0.977273,0.924242,1,0.950413
MCC,0.752618,0.602464,0.53033,0.418121,0.839719,0.651515,0.767649,0.699206,0.471405,0.636364
Precision,0.877622,0.811111,0.78125,0.70979,0.926421,0.826087,0.897516,0.872464,0.805556,0.818182
Recall,0.875,0.791667,0.75,0.708333,0.913043,0.826087,0.869565,0.826087,0.681818,0.818182
F1,0.874783,0.78836,0.742857,0.707826,0.912714,0.826087,0.868075,0.822074,0.645977,0.818182


In [5]:
popphy_stat_df.mean(1)

AUC          0.913034
MCC          0.636939
Precision    0.832600
Recall       0.805978
F1           0.800693
dtype: float64

In [6]:
network, tree_scores = generate_network(g, feature_scores, label_set)
net_json = json.dumps(network, sort_keys=True, indent=4, separators=(',', ': '))

In [7]:
cnn1d_stat_df = pd.DataFrame(index=["AUC", "MCC", "Precision", "Recall", "F1"], columns=cv_list)
mlpnn_stat_df = pd.DataFrame(index=["AUC", "MCC", "Precision", "Recall", "F1"], columns=cv_list)
rf_stat_df = pd.DataFrame(index=["AUC", "MCC", "Precision", "Recall", "F1"], columns=cv_list)
svm_stat_df = pd.DataFrame(index=["AUC", "MCC", "Precision", "Recall", "F1"], columns=cv_list)
lasso_stat_df = pd.DataFrame(index=["AUC", "MCC", "Precision", "Recall", "F1"], columns=cv_list)

run = 0
for seed in seeds:
    skf = StratifiedKFold(n_splits=num_test, shuffle=True, random_state=seed)
    fold = 0
    for train_index, test_index in skf.split(my_maps, labels):
        train_x, test_x = raw_x[train_index,:], raw_x[test_index,:]
        train_y_oh, test_y_oh = labels_oh[train_index,:], labels_oh[test_index,:]
        train_y, test_y = labels[train_index], labels[test_index]
        
        train_x = np.log(train_x + 1)
        test_x = np.log(test_x + 1)
        
        c_prob = [0] * len(np.unique(labels))
        train_weights = []

        for l in np.unique(labels):
            a = float(len(labels))
            b = 2.0 * float((np.sum(labels==l)))
            c_prob[int(l)] = a/b

        c_prob = np.array(c_prob).reshape(-1)

        for l in np.argmax(train_y_oh, 1):
            train_weights.append(c_prob[int(l)])
        train_weights = np.array(train_weights)
        
        scaler = MinMaxScaler().fit(train_x)
        train_x = np.clip(scaler.transform(train_x), 0, 1)
        test_x = np.clip(scaler.transform(test_x), 0, 1) 

        train_oh = [train_x, train_y_oh]
        test_oh = [test_x, test_y_oh]

        train = [train_x, train_y]
        test = [test_x, test_y]
        
        cnn1D_model = CNN1D(train_x.shape[1], num_class, config)
        mlpnn_model = MLPNN(train_x.shape[1], num_class, config)
        rf_model = RF(config)
        svm_model = SVM(config, label_set)
        lasso_model = LASSO(config, label_set)
        
        if fold + run == 0:
            print("CNN-1D")
            print(cnn1D_model.model.summary())
            print("\n\nMLPNN")
            print(mlpnn_model.model.summary())
            print("\n\n Run\tFold\tRF %s\t\tSVM %s\t\tLASSO %s\tMLPNN %s\tCNN-1D %s" % (metric, metric, 
                                                                                   metric, metric, metric))

        cnn1D_model.train(train_oh, train_weights)
        preds, cnn1d_stats = cnn1D_model.test(test_oh)
        if num_class == 2:
                cnn1d_stat_df.loc["AUC"]["Run_" + str(run) + "_CV_" + str(fold)]=cnn1d_stats["AUC"]
        cnn1d_stat_df.loc["MCC"]["Run_" + str(run) + "_CV_" + str(fold)]=cnn1d_stats["MCC"]
        cnn1d_stat_df.loc["Precision"]["Run_" + str(run) + "_CV_" + str(fold)]=cnn1d_stats["Precision"]
        cnn1d_stat_df.loc["Recall"]["Run_" + str(run) + "_CV_" + str(fold)]=cnn1d_stats["Recall"]
        cnn1d_stat_df.loc["F1"]["Run_" + str(run) + "_CV_" + str(fold)]=cnn1d_stats["F1"]

        mlpnn_model.train(train_oh, train_weights)
        preds, mlpnn_stats = mlpnn_model.test(test_oh)
        if num_class == 2:
                mlpnn_stat_df.loc["AUC"]["Run_" + str(run) + "_CV_" + str(fold)]=mlpnn_stats["AUC"]
        mlpnn_stat_df.loc["MCC"]["Run_" + str(run) + "_CV_" + str(fold)]=mlpnn_stats["MCC"]
        mlpnn_stat_df.loc["Precision"]["Run_" + str(run) + "_CV_" + str(fold)]=mlpnn_stats["Precision"]
        mlpnn_stat_df.loc["Recall"]["Run_" + str(run) + "_CV_" + str(fold)]=mlpnn_stats["Recall"]
        mlpnn_stat_df.loc["F1"]["Run_" + str(run) + "_CV_" + str(fold)]=mlpnn_stats["F1"]
        
        rf_model.train(train)
        preds, rf_stats = rf_model.test(test)
        if num_class == 2:
                rf_stat_df.loc["AUC"]["Run_" + str(run) + "_CV_" + str(fold)]=rf_stats["AUC"]
        rf_stat_df.loc["MCC"]["Run_" + str(run) + "_CV_" + str(fold)]=rf_stats["MCC"]
        rf_stat_df.loc["Precision"]["Run_" + str(run) + "_CV_" + str(fold)]=rf_stats["Precision"]
        rf_stat_df.loc["Recall"]["Run_" + str(run) + "_CV_" + str(fold)]=rf_stats["Recall"]
        rf_stat_df.loc["F1"]["Run_" + str(run) + "_CV_" + str(fold)]=rf_stats["F1"]
        
        svm_model.train(train)
        preds, svm_stats = svm_model.test(test)
        if num_class == 2:
                svm_stat_df.loc["AUC"]["Run_" + str(run) + "_CV_" + str(fold)]=svm_stats["AUC"]
        svm_stat_df.loc["MCC"]["Run_" + str(run) + "_CV_" + str(fold)]=svm_stats["MCC"]
        svm_stat_df.loc["Precision"]["Run_" + str(run) + "_CV_" + str(fold)]=svm_stats["Precision"]
        svm_stat_df.loc["Recall"]["Run_" + str(run) + "_CV_" + str(fold)]=svm_stats["Recall"]
        svm_stat_df.loc["F1"]["Run_" + str(run) + "_CV_" + str(fold)]=svm_stats["F1"]
        
        lasso_model.train(train)
        preds, lasso_stats = lasso_model.test(test)
        if num_class == 2:
                svm_stat_df.loc["AUC"]["Run_" + str(run) + "_CV_" + str(fold)]=lasso_stats["AUC"]
        lasso_stat_df.loc["MCC"]["Run_" + str(run) + "_CV_" + str(fold)]=lasso_stats["MCC"]
        lasso_stat_df.loc["Precision"]["Run_" + str(run) + "_CV_" + str(fold)]=lasso_stats["Precision"]
        lasso_stat_df.loc["Recall"]["Run_" + str(run) + "_CV_" + str(fold)]=lasso_stats["Recall"]
        lasso_stat_df.loc["F1"]["Run_" + str(run) + "_CV_" + str(fold)]=lasso_stats["F1"]
        
                          
        
        if metric == "AUC":
                print("# %d\t%d\t%.3f\t\t%.3f\t\t%.3f\t\t%.3f\t\t%.3f" % (run, fold, rf_stats["AUC"], svm_stats["AUC"],
                                                          lasso_stats["AUC"], mlpnn_stats["AUC"], cnn1d_stats["AUC"]))
        if metric == "MCC":
                print("# %d\t%d\t%.3f\t\t%.3f\t\t%.3f\t\t%.3f\t\t%.3f" % (run, fold, rf_stats["MCC"], svm_stats["MCC"], 
                                                          lasso_stats["MCC"], mlpnn_stats["MCC"], cnn1d_stats["MCC"]))

        cnn1D_model.destroy()
        mlpnn_model.destroy()
        del(rf_model)
        del(svm_model)
        del(lasso_model)
        
        fold += 1
    run += 1

CNN-1D
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise (GaussianNois (None, 1, 269, 1)         0         
_________________________________________________________________
conv_0 (Conv2D)              (None, 1, 260, 32)        352       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 130, 32)        0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 1, 121, 32)        10272     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 60, 32)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1920)        